In [ ]:
import os
import pyaudio
import wave
import speech_recognition as sr
import threading
from transformers import pipeline

# Set the TOKENIZERS_PARALLELISM environment variable to false
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Parameters for recording
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 20
WAVE_OUTPUT_FILENAME = "output.wav"

# Initialize recognizer
recognizer = sr.Recognizer()

def record_audio():
    audio = pyaudio.PyAudio()

    # Start recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("Recording...")
    frames = []

    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    
    print("Finished recording.")

    # Stop recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

def transcribe_audio():
    with sr.AudioFile(WAVE_OUTPUT_FILENAME) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print("Transcription: " + text)
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
            return None

def summarize_text(text):
    summarizer = pipeline("summarization", model='sshleifer/distilbart-cnn-12-6', device=0)
    # Adjust max_length and min_length based on the length of the input text
    max_length = min(50, len(text.split()) // 2)
    min_length = min(10, max_length - 1)
    summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    print("Summary: " + summary[0]['summary_text'])

def sentiment_analysis(text):
    classifier = pipeline("sentiment-analysis", model='distilbert-base-uncased-finetuned-sst-2-english', device=0)
    result = classifier(text)
    print("Sentiment: " + result[0]['label'])

# Example usage
if __name__ == "__main__":
    record_audio()
    transcribed_text = transcribe_audio()
    if transcribed_text:
        # Create threads for summarization and sentiment analysis
        summarize_thread = threading.Thread(target=summarize_text, args=(transcribed_text,))
        sentiment_thread = threading.Thread(target=sentiment_analysis, args=(transcribed_text,))

        # Start threads
        summarize_thread.start()
        sentiment_thread.start()

        # Wait for threads to complete
        summarize_thread.join()
        sentiment_thread.join()

Recording...
Finished recording.
Transcription: today I went to the movies with the kids
Sentiment: POSITIVE
Summary:  today
